In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import FindMaxEpsForAllIneqs, MosekSolver, MosekSolverDetails
from time import time

# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


# Setup IRIS Options and Generate Regions

In [58]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9],
                          [-0.6,-1.3,-0.3],
                          [-0.9,-1.7,-1.3],
                          [1.30, 1.50, -0.1],
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)
def check_empty_polytope(C, d):
    prog = MathematicalProgram()
    t = prog.NewContinuousVariables(C.shape[1], "t")

    prog.AddLinearConstraint(C,-np.inf*np.ones(C.shape[0]), d, t)
    prog.AddBoundingBoxConstraint(limits_t[0], limits_t[1], t)
    result = MosekSolver().Solve(prog)
    return result.is_success(), result.get_solution_result()

In [11]:
# RUN THIS CELL IF YOU WANT TO DO SNOPT IRIS

# regions = []

# iris_options = IrisOptionsRationalSpace()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 20
# iris_options.configuration_space_margin = 1e-5
# iris_options.max_faces_per_collision_pair = 60
# iris_options.termination_threshold = -1
# iris_options.q_star = np.zeros(3)
# iris_options.relative_termination_threshold = 0.05
# iris_options.enable_ibex = False
# #deprecated
# iris_options.certify_region_with_sos_during_generation = False
# iris_options.certify_region_with_sos_after_generation = False

# for i, s in enumerate(seed_points):
#     plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
#     if False:
#         #starting_hpolyhedron = regions[i-1]
#         r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
#                                               iris_options, starting_hpolyhedron)
#     else:
#         r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
#     regions.append(r)
#     print(f'Completed region: {i+1}/{len(seed_points)}')
#     print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
#     print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
#     print()

In [12]:
# if do_viz:
#     visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

## Create default region

In [59]:
regions_default = []
max_vects = []
min_vects = []
vector_bisection_options_list = []
bilinear_alteration_options_list = []
interleaved_options_list = []

#parameters for creating a default region. Currently set to create a box
num_perm_dim = 0
num_rot = 1

for i, s in enumerate(seed_points):
    regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
    
    eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1], s)
    
    min_vects.append(eps_min_vect)
    
    eps_max_vect = FindMaxEpsForAllIneqs(plant, plant.GetMyMutableContextFromRoot(context), 
                                q_star, regions_default[i].A(), regions_default[i].b(), 
                                         eps_min_vect, limits_t[0], limits_t[1], s)
    max_vects.append(eps_max_vect)
    
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 10
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = eps_min_vect
    vector_bisection_search_options.epsilon_max = eps_max_vect
    vector_bisection_search_options.num_threads = -1
    vector_bisection_options_list.append(vector_bisection_search_options)
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 0
    bilinear_alternation_options.polytope_backoff_scale = 0
    bilinear_alternation_options.num_threads = -1
    bilinear_alteration_options_list.append(bilinear_alternation_options)
    
    interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
    interleaved_options.vector_bisection_search_options = vector_bisection_search_options
    interleaved_options.bilinear_alternation_options = bilinear_alternation_options
    #actually does round robin
    interleaved_options.use_vector_bisection_search = True
    interleaved_options.max_method_switch = 1
    interleaved_options_list.append(interleaved_options)
    
    
num_round_robin_rounds = 1

if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
    

In [50]:
i = 2
s = seed_points[i,:]
C, d = regions_default[i].A(), regions_default[i].b()
vector_bisection_search_options = vector_bisection_options_list[i]
interleaved_options = interleaved_options_list[i]

In [51]:
C.shape

(6, 3)

In [41]:
print(vector_bisection_search_options.epsilon_max)
print(vector_bisection_search_options.epsilon_min)
print(vector_bisection_search_options.epsilon_max-
     vector_bisection_search_options.epsilon_min)

[ 0.05004171 -0.09999999  0.73417852  0.05004171  0.53309677 -0.10000001]
[-0.09999999 -0.09999999 -0.09999999 -0.10000001 -0.10000001 -0.10000001]
[ 1.50041698e-01 -4.44089210e-16  8.34178507e-01  1.50041718e-01
  6.33096784e-01  5.55111512e-17]


In [42]:
# #plot the new polytopes which each individual face moved
# if do_viz:
#     for i, r in enumerate(regions_default):
#         C, d = r.A(), r.b()
#         eps_max = vector_bisection_options_list[i].epsilon_max
#         r_moved_list = []
#         for j in range(len(d)):
#             d_new = d.copy()
#             d_new[j] += eps_max[j]
#             r_moved_list.append(HPolyhedron(C, d_new))
#         if do_viz:
#             visualizer.plot_regions(r_moved_list, ellipses=None, region_suffix=f'_default_{i}_maxed')
#         if do_viz:
#             visualizer.plot_regions([HPolyhedron(C, d+0.99*eps_max)], ellipses=None, region_suffix=f'_all_{i}_maxed')

## Run Cspace Free Region

In [60]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [52]:
t0 = time.time()
cspace_free_region_solution_interleaved = cspace_free_region.InterleavedCSpacePolytopeSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, num_round_robin_rounds, 
                                                                     interleaved_options,
                                                                     solver_options, s)
t1 = time.time()


[2022-02-09 00:07:16.132] [console] [info] Starting bisections
[2022-02-09 00:07:18.430] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:07:19.593] [console] [info] bilinear alt on d succeeded
[2022-02-09 00:07:19.608] [console] [info] Ineq 1/6 in round 1
[2022-02-09 00:07:19.609] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 00:07:19.617] [console] [info] output of eps_redundant_max is 1.8224695040938002
[2022-02-09 00:07:19.618] [console] [info] current gap is epsilon_gap=0.8341784992408996
[2022-02-09 00:07:21.427] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:07:22.660] [console] [info] bilinear alt on d succeeded
[2022-02-09 00:07:22.677] [console] [info] epsilon=0.3170892670967813 is feasible
[2022-02-09 00:07:22.677] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 00:07:22.685] 

[2022-02-09 00:07:55.897] [console] [info] epsilon=-0.024665965388246513 is infeasible
[2022-02-09 00:07:55.897] [console] [info] current gap is epsilon_gap=0.0747076737637853
[2022-02-09 00:07:57.739] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004700bd8, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-09 00:07:57.740] [console] [info] epsilon=-0.06201980227013917 is infeasible
[2022-02-09 00:07:57.740] [console] [info] current gap is epsilon_gap=0.03735383688189265
[2022-02-09 00:07:59.588] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:08:00.871] [console] [info] bilinear alt on d succeeded
[2022-02-09 00:08:00.889] [console] [info] epsilon=-0.0806967207110855 is feasible
[2022-02-09 00:08:00.890] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 00:08:00.898] [console] [info] output of eps_redundant_max is 1.

[2022-02-09 00:08:27.998] [console] [info] current gap is epsilon_gap=0.016410799877457485
[2022-02-09 00:08:29.844] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-09 00:08:29.845] [console] [info] epsilon=-0.07303929070539242 is infeasible
[2022-02-09 00:08:29.845] [console] [info] current gap is epsilon_gap=0.00820539993872875
[2022-02-09 00:08:31.731] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:08:33.059] [console] [info] bilinear alt on d succeeded
[2022-02-09 00:08:33.073] [console] [info] epsilon=-0.0771419906747568 is feasible
[2022-02-09 00:08:33.074] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 00:08:33.081] [console] [info] output of eps_redundant_max i

[2022-02-09 00:11:00.991] [console] [info] cost improvement 0.0011191400470564328
[2022-02-09 00:11:02.246] [console] [info] Iter: 10, polytope step cost -0.004940738339393831, solver time 0.3309361934661865
[2022-02-09 00:11:16.887] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:11:16.888] [console] [info] Lagrangian step time 14.625 s
[2022-02-09 00:11:16.907] [console] [info] max(power(det(P), 1/4))=0.2530851320398552, solver_time 0.010756969451904297
[2022-02-09 00:11:16.908] [console] [info] cost improvement 0.001076334570370041
[2022-02-09 00:11:18.305] [console] [info] Iter: 11, polytope step cost -0.004803616059098791, solver time 0.42306017875671387
[2022-02-09 00:11:29.949] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:11:29.950] [console] [info] Lagrangian step time 11.628 s
[2022-02-09 00:11:29.987] [console] [info] max(power(det(P), 1/4))=0.25411964644636853, solver_time 0.023354053497314453
[2022-02-09

[2022-02-09 00:12:23.762] [console] [info] epsilon=0.0076288078184245205 is feasible
[2022-02-09 00:12:23.763] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 00:12:23.770] [console] [info] output of eps_redundant_max is 0.9183393805397282
[2022-02-09 00:12:23.772] [console] [info] reset eps_min=0.008302824655234214, eps_max=0.7341785167172311
[2022-02-09 00:12:23.772] [console] [info] Ineq 2/6 in round 1
[2022-02-09 00:12:23.778] [console] [info] output of eps_redundant_max is 2.032412028644645
[2022-02-09 00:12:23.779] [console] [info] current gap is epsilon_gap=1.2091319234011464
[2022-02-09 00:12:25.580] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:12:26.849] [console] [info] bilinear alt on d succeeded
[2022-02-09 00:12:26.866] [console] [info] epsilon=-0.07146918770845595 is feasible
[2022-02-09 00:12:26.866] [console] [info] epsilon exceeds components of epsilon max in some com

...

[2022-02-09 00:15:58.229] [console] [info] max(power(det(P), 1/4))=0.24360584330516064, solver_time 0.008342981338500977
[2022-02-09 00:15:58.230] [console] [info] cost improvement 0.001301405001834649
[2022-02-09 00:15:59.489] [console] [info] Iter: 10, polytope step cost -0.0045132248431117164, solver time 0.2821941375732422
[2022-02-09 00:16:10.471] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:16:10.473] [console] [info] Lagrangian step time 10.968 s
[2022-02-09 00:16:10.522] [console] [info] max(power(det(P), 1/4))=0.24484604185323705, solver_time 0.026835203170776367
[2022-02-09 00:16:10.523] [console] [info] cost improvement 0.0012401985480764155
[2022-02-09 00:16:11.887] [console] [info] Iter: 11, polytope step cost -0.004357335019722837, solver time 0.3720400333404541
[2022-02-09 00:16:25.137] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:16:25.138] [console] [info] Lagrangian step time 13.235 s
[2022-02-0

[2022-02-09 00:17:41.838] [console] [info] epsilon=0.07076882291855764 is infeasible
[2022-02-09 00:17:41.838] [console] [info] current gap is epsilon_gap=0.09477281339981049
[2022-02-09 00:17:43.621] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box4_Collision@00000000046fbfd8, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(box_scene::box4_Collision@00000000046fbfd8, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-09 00:17:43.622] [console] [info] epsilon=0.023382416218652398 is infeasible
[2022-02-09 00:17:43.622] [console] [info] current gap is epsilon_gap=0.047386406699905244
[2022-02-09 00:17:45.474] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:17:46.670] [console] [info] bilinear alt on d succeeded
[2022-02-09 00:17:46.686] [console] [info] epsilon=-0.00031078713130022403 is feasible
[2022-02-09 00:17:46.686] [console] [info] epsilon exceeds components of epsilon max in some components, increa

[2022-02-09 00:18:09.049] [console] [info] epsilon=0.2719897283358421 is infeasible
[2022-02-09 00:18:09.050] [console] [info] current gap is epsilon_gap=0.261107045656275
[2022-02-09 00:18:11.077] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-09 00:18:11.079] [console] [info] epsilon=0.1414362055077046 is infeasible
[2022-02-09 0

[2022-02-09 00:18:41.644] [console] [info] epsilon=0.0029398207432379817 is infeasible
[2022-02-09 00:18:41.645] [console] [info] current gap is epsilon_gap=0.003140125842153383
[2022-02-09 00:18:43.608] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-09 00:18:43.609] [console] [info] epsilon=0.00136975782216129 is infeasible
[2022-02-09 00:18:43.609] [console] [info] current gap is epsilon_gap=0.0015700629210766916
[2022-02-09 00:18:45.463] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:18:46.771] [console] [info] bilinear alt on d succeeded
[2022-02-09 00:18:46.788] [console] [info] epsilon=0.0005847263616229443 is feasible
[2022-02-09 00:18:46.789] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 00:18:46.803] [console] [info] output of eps_redundant_max 

[2022-02-09 00:21:07.682] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:21:07.683] [console] [info] Lagrangian step time 11.591 s
[2022-02-09 00:21:07.695] [console] [info] max(power(det(P), 1/4))=0.24265839905056003, solver_time 0.005257129669189453
[2022-02-09 00:21:07.696] [console] [info] cost improvement 0.0013698879027291
[2022-02-09 00:21:09.041] [console] [info] Iter: 9, polytope step cost -0.004714635077148964, solver time 0.36413097381591797
[2022-02-09 00:21:20.886] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 00:21:20.887] [console] [info] Lagrangian step time 11.831 s
[2022-02-09 00:21:20.899] [console] [info] max(power(det(P), 1/4))=0.24397185036947955, solver_time 0.005400180816650391
[2022-02-09 00:21:20.900] [console] [info] cost improvement 0.001313451318919523
[2022-02-09 00:21:22.294] [console] [info] Iter: 10, polytope step cost -0.004533380425126114, solver time 0.35570287704467773
[2022-02-09 0

In [ ]:
print(f"Completed points in {t1-t0}s")

In [45]:
r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_interleaved.C]),
                np.hstack([starting_poly.b(), cspace_free_region_solution_interleaved.d])
                )
e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_interleaved.P), 
                   cspace_free_region_solution_interleaved.q)
if do_viz:
    visualizer.plot_regions([r], 
                            ellipses=[e], region_suffix='_interleaved_single')

In [23]:
# r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_round_robin.C]),
#                 np.hstack([starting_poly.b(), cspace_free_region_solution_round_robin.d])
#                 )
# e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_round_robin.P), 
#                    cspace_free_region_solution_round_robin.q)
# if do_viz:
#     visualizer.plot_regions([r], 
#                             ellipses=[e], region_suffix='_round_robin_single')

In [61]:
seed_point_list = [s for s in seed_points] 

C_mats = [r.A() for r in regions_default]
d_vects = [r.b() for r in regions_default]
t0 = time.time()
cspace_free_region_solution_interleaved_vect = cspace_free_region.InterleavedCSpacePolytopeSearchForSeedPoints(
                                                                    q_star,
                                                                     filtered_collision_pairs,
                                                                     C_mats, d_vects, num_round_robin_rounds, 
                                                                     interleaved_options_list,
                                                                     solver_options, seed_point_list)
# cspace_free_region_solution_round_robin_vect = cspace_free_region.CspacePolytopeRoundRobinBisectionSearchForSeedPoints(
#                                                                     q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C_mats, d_vects, num_round_robin_rounds, 
#                                                                      vector_bisection_options_list,
#                                                                      solver_options, seed_point_list)
t1 = time.time()


[2022-02-09 07:49:00.199] [console] [info] Starting bisections
[2022-02-09 07:49:00.199] [console] [info] Starting bisections
[2022-02-09 07:49:00.199] [console] [info] Starting bisections
[2022-02-09 07:49:00.199] [console] [info] Starting bisections
[2022-02-09 07:49:00.199] [console] [info] Starting bisections
[2022-02-09 07:49:00.199] [console] [info] Starting bisections
[2022-02-09 07:49:00.199] [console] [info] Starting bisections
[2022-02-09 07:49:00.200] [console] [info] Starting bisections
[2022-02-09 07:49:25.030] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:49:25.094] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:49:25.140] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:49:25.150] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:49:25.153] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:49:25.153] [cons

[2022-02-09 07:49:53.593] [console] [info] epsilon=0.2207494126502239 is infeasible
[2022-02-09 07:49:53.595] [console] [info] current gap is epsilon_gap=0.32074940174053806
[2022-02-09 07:49:56.455] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:49:56.483] [console] [info] epsilon=0.2776388233497269 is feasible
[2022-02-09 07:49:56.484] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 07:49:56.486] [console] [info] output of eps_redundant_max is 2.295405765498081
[2022-02-09 07:49:56.497] [console] [info] reset eps_min=0.30033467238526035, eps_max=0.6552776476118607
[2022-02-09 07:49:56.498] [console] [info] current gap is epsilon_gap=0.35494297522660034
[2022-02-09 07:49:57.801] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:49:58.090] [console] [info] epsilon=0.1801690977239196 is feasible
[2022-02-09 07:49:58.091] [console] [info] epsilon exceeds components of epsilon max in some components, 

[2022-02-09 07:50:20.658] [console] [info] current gap is epsilon_gap=0.26136367489426426
[2022-02-09 07:50:23.036] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:50:23.067] [console] [info] epsilon=1.2443099070930659 is feasible
[2022-02-09 07:50:23.069] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 07:50:23.071] [console] [info] output of eps_redundant_max is 1.1344144062678456
[2022-02-09 07:50:23.071] [console] [info] reset eps_min=1.2545897783311293, eps_max=1.6918008590777487
[2022-02-09 07:50:23.071] [console] [info] current gap is epsilon_gap=0.4372110807466194
[2022-02-09 07:50:25.043] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:50:25.084] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:50:25.697] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:50:25.956] [console] [warning] Cannot find Lagran

[2022-02-09 07:50:48.219] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:50:48.236] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:50:48.246] [console] [info] epsilon=0.6158917194832696 is feasible
[2022-02-09 07:50:48.247] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 07:50:48.249] [console] [info] output of eps_redundant_max is 1.9782924868929848
[2022-02-09 07:50:48.249] [console] [info] reset eps_min=0.6174479509903568, eps_max=0.6552776476118607
[2022-02-09 07:50:48.249] [console] [info] current gap is epsilon_gap=0.03782969662150393
[2022-02-09 07:50:48.741] [console] [info] epsilon=0.47645291252743593 is feasible
[2022-02-09 07:50:48.744] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 07:50:48.748] [console] [info] output of eps_redundant_max is 1.4472027485455456
[2022-02-09 07:50:48.815] [console] [info] reset eps_min=

[2022-02-09 07:51:14.526] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004700bd8, iiwa7_twoDOF::iiwa_twoDOF_link_6_collision)

[2022-02-09 07:51:14.527] [console] [info] epsilon=0.6460065022889632 is infeasible
[2022-02-09 07:51:14.528] [console] [info] current gap is epsilon_gap=0.009271145322897523
[2022-02-09 07:51:14.750] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:51:15.035] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_l

...

[2022-02-09 07:56:10.115] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:56:10.131] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:56:10.215] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:56:10.224] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:56:10.239] [console] [info] epsilon=-0.014488680796929831 is feasible
[2022-02-09 07:56:10.240] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 07:56:10.245] [console] [info] epsilon=-0.05707995932773713 is feasible
[2022-02-09 07:56:10.246] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 07:56:10.275] [console] [info] output of eps_redundant_max is 0.3352081839398192
[2022-02-09 07:56:10.275] [console] [info] output of eps_redundant_max is 1.733546234147933
[2022-02-09 07:56:10.276] [console] [info] reset eps_min=-0.01

[2022-02-09 07:56:40.041] [console] [info] output of eps_redundant_max is 1.7221129216348179
[2022-02-09 07:56:40.042] [console] [info] reset eps_min=0.0003565999353138949, eps_max=0.0003565999353138949
[2022-02-09 07:56:40.042] [console] [info] Ineq 5/6 in round 1
[2022-02-09 07:56:40.044] [console] [info] output of eps_redundant_max is 0.4219401488469786
[2022-02-09 07:56:40.044] [console] [info] Ineq 6/6 in round 1
[2022-02-09 07:56:40.045] [console] [info] output of eps_redundant_max is 1.0241125760259702
[2022-02-09 07:56:40.554] [console] [info] max(power(det(P), 1/4))=0.1647712161483324, solver_time 0.5073881149291992
[2022-02-09 07:56:40.605] [console] [info] Starting Alternations
[2022-02-09 07:56:41.053] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:56:41.140] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF

[2022-02-09 07:57:03.615] [console] [info] output of eps_redundant_max is 0.4260645869703046
[2022-02-09 07:57:03.617] [console] [info] reset eps_min=0.0017045806342335101, eps_max=0.0017045806342335101
[2022-02-09 07:57:03.618] [console] [info] Ineq 6/6 in round 1
[2022-02-09 07:57:03.619] [console] [info] output of eps_redundant_max is 0.8286593036527976
[2022-02-09 07:57:03.648] [console] [info] max(power(det(P), 1/4))=0.14813862069320996, solver_time 0.027012109756469727
[2022-02-09 07:57:03.759] [console] [info] Starting Alternations
[2022-02-09 07:57:03.770] [console] [info] mosek info 5, InfeasibleConstraints
[2022-02-09 07:57:03.771] [console] [warning] Failed to find the polytope at iteration 0
[2022-02-09 07:57:03.772] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box1_Collision@0000000004700728, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-09 07:57:03.773] [console] [info] epsilon=0.24928023110036737 is infeasible
[2022

[2022-02-09 07:57:32.859] [console] [info] bilinear alt on d succeeded
[2022-02-09 07:57:32.877] [console] [info] epsilon=-0.000437633475607508 is feasible
[2022-02-09 07:57:32.878] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 07:57:32.903] [console] [info] output of eps_redundant_max is 1.086195722290788
[2022-02-09 07:57:32.904] [console] [info] reset eps_min=0.002178699313190015, eps_max=0.002178699313190015
[2022-02-09 07:57:32.904] [console] [info] Ineq 6/6 in round 1
[2022-02-09 07:57:32.916] [console] [info] output of eps_redundant_max is -2.013342850029288
[2022-02-09 07:57:32.917] [console] [warning] round robin bisection search failed: epsilon max is less than epsilon min.
 difference is = 
 -0.0552959
  0.525811
 0.0021787
  0.561243
   1.41403
-0.0934736
[2022-02-09 07:57:32.934] [console] [info] max(power(det(P), 1/4))=0.3785108152173696, solver_time 0.011614084243774414
[2022-02-09 07:57:32.963] [console

[2022-02-09 07:58:21.847] [console] [info] Iter: 18, polytope step cost -0.011632754139422398, solver time 0.35306787490844727
[2022-02-09 07:58:23.834] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 07:58:23.835] [console] [info] Lagrangian step time 1.97 s
[2022-02-09 07:58:23.846] [console] [info] max(power(det(P), 1/4))=0.4042331388806107, solver_time 0.004889011383056641
[2022-02-09 07:58:23.846] [console] [info] cost improvement 0.0021986945801261704
[2022-02-09 07:58:25.379] [console] [info] Iter: 19, polytope step cost -0.011209448420697264, solver time 0.532404899597168
[2022-02-09 07:58:25.405] [console] [info] max(power(det(P), 1/4))=0.40628145478228933, solver_time 0.004133939743041992


In [65]:
print(len(seed_points))

8


In [62]:
print(seed_points)

[[ 0.          0.          0.        ]
 [ 0.42279322  0.7602044  -0.42279322]
 [ 0.05004171  0.48305507 -0.68413681]
 [ 0.10033467  1.02963856 -0.30933625]
 [ 0.25534192 -1.39838259 -0.48305507]
 [-0.30933625 -0.7602044  -0.15113522]
 [-0.48305507 -1.13833271 -0.7602044 ]
 [ 0.7602044   0.93159646 -0.05004171]]


In [73]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=None, region_suffix='_interleaved2')

Completed points in 565.2634370326996s


In [72]:
visualizer.vis2["regions_interleaved"]

<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x7f71c76cf190> at path: <meshcat.path.Path object at 0x7f718dce99a0>>

# Don't run after here

In [22]:
cspace_free_region_solution_round_robin = cspace_free_region.CspacePolytopeRoundRobinBisectionSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, num_rounds, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-08 19:29:47.629] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:29:49.330] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:29:49.358] [console] [info] Ineq 1/6 in round 1
[2022-02-08 19:29:49.359] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:29:49.379] [console] [info] output of eps_redundant_max is 1.8224695040938002
[2022-02-08 19:29:49.381] [console] [info] current gap is epsilon_gap=0.8341784992408996
[2022-02-08 19:29:52.187] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:29:53.997] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:29:54.033] [console] [info] epsilon=0.3170892670967813 is feasible
[2022-02-08 19:29:54.034] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:29:54.051] [console] [info] output of eps_redundant_max is 1.3733133612699

[2022-02-08 19:30:42.509] [console] [info] epsilon=-0.024665965387383592 is infeasible
[2022-02-08 19:30:42.510] [console] [info] current gap is epsilon_gap=0.07470767376292238
[2022-02-08 19:30:45.547] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@00005642fcbecef8, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-08 19:30:45.549] [console] [info] epsilon=-0.062019802268844786 is infeasible
[2022-02-08 19:30:45.549] [console] [info] current gap is epsilon_gap=0.03735383688146119
[2022-02-08 19:30:48.945] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:30:50.770] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:30:50.810] [console] [info] epsilon=-0.08069672070957538 is feasible
[2022-02-08 19:30:50.811] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:30:50.821] [console] [info] output of eps_redundant_max is

[2022-02-08 19:31:36.939] [console] [info] current gap is epsilon_gap=0.01641079987684163
[2022-02-08 19:31:40.455] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 19:31:40.457] [console] [info] epsilon=-0.07303929070077358 is infeasible
[2022-02-08 19:31:40.458] [console] [info] current gap is epsilon_gap=0.008205399938420815
[2022-02-08 19:31:43.692] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:31:45.558] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:31:45.600] [console] [info] epsilon=-0.07714199066998399 is feasible
[2022-02-08 19:31:45.601] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:31:45.627] [console] [info] output of eps_redundant_max 

In [29]:
cspace_free_region_solution_bilinear = cspace_free_region.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     cspace_free_region_solution_round_robin.C,
                                                                    0.75*cspace_free_region_solution_round_robin.d,
                                                                     bilinear_alternation_options,
                                                                     solver_options, s)

[2022-02-08 19:34:36.721] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:36.723] [console] [info] Lagrangian step time 3.045 s
[2022-02-08 19:34:36.751] [console] [info] max(power(det(P), 1/4))=0.2071381570004865, solver_time 0.018611907958984375
[2022-02-08 19:34:36.753] [console] [info] cost improvement inf
[2022-02-08 19:34:38.615] [console] [info] Iter: 0, polytope step cost -0.008828768121858371, solver time 0.48531293869018555
[2022-02-08 19:34:41.859] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:41.860] [console] [info] Lagrangian step time 3.218 s
[2022-02-08 19:34:41.881] [console] [info] max(power(det(P), 1/4))=0.2105033555416147, solver_time 0.013540029525756836
[2022-02-08 19:34:41.882] [console] [info] cost improvement 0.0033651985411282115
[2022-02-08 19:34:43.780] [console] [info] Iter: 1, polytope step cost -0.008522369595631428, solver time 0.532606840133667
[2022-02-08 19:34:46.920] [console

[2022-02-08 19:36:00.640] [console] [info] Iter: 16, polytope step cost -0.00557567747292513, solver time 0.38878607749938965
[2022-02-08 19:36:04.137] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:04.141] [console] [info] Lagrangian step time 3.472 s
[2022-02-08 19:36:04.172] [console] [info] max(power(det(P), 1/4))=0.24350116763464993, solver_time 0.01593494415283203
[2022-02-08 19:36:04.174] [console] [info] cost improvement 0.001400599503426303
[2022-02-08 19:36:05.966] [console] [info] Iter: 17, polytope step cost -0.005466929346269455, solver time 0.3860509395599365
[2022-02-08 19:36:09.113] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:09.114] [console] [info] Lagrangian step time 3.119 s
[2022-02-08 19:36:09.142] [console] [info] max(power(det(P), 1/4))=0.24485582644352566, solver_time 0.00913095474243164
[2022-02-08 19:36:09.145] [console] [info] cost improvement 0.001354658808875725
[2022-02-08 19:3

In [31]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_bilinear.C,
                                         cspace_free_region_solution_bilinear.d)], 
                            ellipses=None, region_suffix='_bilinear')

In [15]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_round_robin_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_round_robin')

In [22]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution.C, 0.5*cspace_free_region_solution.d)], 
                            ellipses=None, region_suffix='_round_robin')

In [26]:
cspace_free_region_solution_vector_bisection = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-07 15:27:37.125] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-07 15:27:37.194] [console] [info] max(power(det(P), 1/4))=-nan, solver_time 0.05913114547729492
[2022-02-07 15:27:40.385] [console] [info] bilinear alt on d succeeded
[2022-02-07 15:27:40.411] [console] [info] max(power(det(P), 1/4))=1.195363453968912e-06, solver_time 0.014796972274780273
[2022-02-07 15:27:45.796] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-07 15:27:45.797] [console] [info] total iter=1, feasible iter = 0
[2022-02-07 15:27:51.001] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_

In [29]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_vector_bisection.C, 
                                         cspace_free_region_solution_vector_bisection.d)], 
                            ellipses=None, region_suffix='_vector_bisection')

(True, <SolutionResult.kSolutionFound: 0>)


In [96]:
i

2